In [23]:
import numpy as np
from susy_qm import create_matrix
from functools import reduce

In [460]:
# Parameters
N = 1   # Number of sites
cutoff = 2    # Basis cutoff
a = 1.0       # Lattice spacing

In [461]:
def create_shift_matrix(num_sites, direction):
 
    S = np.zeros((num_sites, num_sites), dtype=np.complex128)
    for i in range(num_sites - 1):
        if direction == 'forward':
            S[i, i + 1] = 1
        elif direction == 'backward':
            S[i + 1, i] = 1
    return S

In [462]:
def tensor_identity(cut_off, site, total_sites, operator):

    I = np.eye(cut_off, dtype=np.complex128)
    operators = [I] * total_sites
    operators[site] = operator
    
    return reduce(np.kron, operators) 

In [465]:
create_matrix(cutoff, 'q')

array([[0.        +0.j, 0.70710678+0.j],
       [0.70710678+0.j, 0.        +0.j]])

In [466]:
tensor_identity(cutoff, n, N, q)

array([[0.        +0.j, 0.70710678+0.j],
       [0.70710678+0.j, 0.        +0.j]])

In [469]:
# Initialize the Hamiltonian
I_b = np.eye(cutoff ** N)
I_f = np.eye(2 ** N)

#H = np.zeros(np.kron(I_b,I_f).shape, dtype=np.complex128)

q = create_matrix(cutoff, 'q')
p = create_matrix(cutoff, 'p')

for n in range(N):

    q_n = tensor_identity(cutoff, n, N, q)
    p_n = tensor_identity(cutoff, n, N, p)
    
    # Kinetic term
    p2 = np.matmul(p_n,p_n) #/ (2 * a)
    kinetic_term = np.kron(I_f,p2)
    
    # Potential term
    W_prime = q_n  # W'(q) = q
    W_double_prime = I_b #W''(q) = 1
    potential = np.matmul(W_prime,W_prime)#*(a / 2)
    potential_term = np.kron(I_f, potential)

    # Commutator term [b^†, b] = -Z
    Z = np.array([[1, 0], [0, -1]])  # Pauli Z matrix for fermion number
    commutator_term = np.kron(Z, W_double_prime)

    H = 0.5 * (kinetic_term + potential_term + commutator_term)

    '''
    # Gradient and interaction terms
    
    if 0 < n < N - 1:
        print('a')
        q_np1 = tensor_identity(cutoff, (n + 1) % N, N, q)  # (n+1)-th site (periodic boundary)
        q_nm1 = tensor_identity(cutoff, (n - 1) % N, N, q)  # (n-1)-th site (periodic boundary)
    
        gradient = (q_np1 - q_nm1) / (2 * a)
        H += (a / 2) * np.matmul(gradient,gradient)
        H += a * np.matmul(V,gradient)
    
    # Fermionic terms
    if 0 < n < N - 1:
        print('b')
        fermion_term = (-1) ** n * VP @ (np.matmul(tensor_identity(cutoff, n, N, S_dagger), tensor_identity(cutoff, n, N, S)) - 0.5 * I_b)
        H += fermion_term
    
        # Fermionic hopping
        #fermion_hopping = (np.matmul(tensor_identity(cutoff, n, N, np.eye(cutoff)), tensor_identity(cutoff, n + 1, N, np.eye(cutoff))) + np.matmul(tensor_identity(cutoff, n, N, np.eye(cutoff)), tensor_identity(cutoff, n + 1, N, np.eye(cutoff))))
        #H += fermion_hopping / (2 * a)
    
    '''


In [470]:
np.sort(np.linalg.eig(H)[0].real)[:7]

array([-1.11022302e-16, -1.11022302e-16,  1.00000000e+00,  1.00000000e+00])

In [ ]:
##def construct_hamiltonian(N, cutoff, a, V, dV):
       
# Initialize the Hamiltonian
dim = cutoff ** N  # Total Hilbert space dimension
H = np.zeros((dim, dim), dtype=np.complex128)

for n in range(N):

    # Bosonic terms
    I_b = np.eye(dim)

    q_n = tensor_identity(cutoff, n, N, q)
    p_n = tensor_identity(cutoff, n, N, p)
    
    # Kinetic term
    H += np.matmul(p_n,p_n) / (2 * a)
    
    # Potential term
    c= -0.2
    V = q_n#c + q_n**2 # 
    VP = I_b#0.5*q_n#
    H += (a / 2) * np.matmul(V,V)

    # Subtract the constant offset
    H -= 0.5 * I_b

    # Gradient and interaction terms
    '''
    if 0 < n < N - 1:
        print('a')
        q_np1 = tensor_identity(cutoff, (n + 1) % N, N, q)  # (n+1)-th site (periodic boundary)
        q_nm1 = tensor_identity(cutoff, (n - 1) % N, N, q)  # (n-1)-th site (periodic boundary)
    
        gradient = (q_np1 - q_nm1) / (2 * a)
        H += (a / 2) * np.matmul(gradient,gradient)
        H += a * np.matmul(V,gradient)
    '''
    # Fermionic terms
    if 0 < n < N - 1:
        print('b')
        fermion_term = (-1) ** n * VP @ (np.matmul(tensor_identity(cutoff, n, N, S_dagger), tensor_identity(cutoff, n, N, S)) - 0.5 * I_b)
        H += fermion_term
    
        # Fermionic hopping
        #fermion_hopping = (np.matmul(tensor_identity(cutoff, n, N, np.eye(cutoff)), tensor_identity(cutoff, n + 1, N, np.eye(cutoff))) + np.matmul(tensor_identity(cutoff, n, N, np.eye(cutoff)), tensor_identity(cutoff, n + 1, N, np.eye(cutoff))))
        #H += fermion_hopping / (2 * a)
    
#return H


In [363]:
# Display the Hamiltonian
print("Hamiltonian shape:", H.shape)
print(H.real)


Hamiltonian shape: (8, 8)
[[-1.11022302e-16  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  2.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   4.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  6.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.000000

In [364]:
np.sort(np.linalg.eig(H)[0].real)[:7]

array([-1.11022302e-16,  1.00000000e+00,  2.00000000e+00,  3.00000000e+00,
        3.00000000e+00,  4.00000000e+00,  5.00000000e+00])